In [1]:
import json
from datasets import load_dataset, Dataset

# Specify the path to the JSON file
file_path = 'datasets/squad-sr-lat.json'

# Open the JSON file and load its contents
with open(file_path, 'r', encoding='utf8') as file:
    data = json.load(file)

# Now, 'data' contains the parsed JSON data as a Python dictionary
print(data.keys())


dict_keys(['version', 'data'])


In [2]:
i = 1
titles = []
contexts = []
queryes = []
for article in data['data']:
    title = article['title']
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        currect_queryes = []
        for qa_pair in paragraph['qas']:
            question = qa_pair['question']
            
            currect_queryes.append(question)
            i += 1
        titles.append(title)
        contexts.append(context)
        queryes.append(currect_queryes)
                # print(answer)

In [3]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queryes})

dataset_for_eval_df.head()
dataset_for_eval_df['queries'][0]

['Kome se u 1858. godini u Lurdu, Francuska, navodno pojavila Deva Marija?',
 'Šta je ispred Glavne zgrade Notr Dame?',
 'Bazilika Svetog srca u Notr Dami je pored koje strukture?',
 'Šta je Grota u Notr Dami?',
 'Šta se nalazi na vrhu Glavne zgrade u Notr Damu?']

In [4]:
import pickle
import pyarrow as pa
import pyarrow.parquet as pq

dataset_for_eval_df.to_csv('datasets/squad_processed.csv', index=False)

table = pa.Table.from_pandas(dataset_for_eval_df)
pq.write_table(table, 'processed_datasets/squad_processed.parquet')


ms_marco

In [5]:
from datasets import load_dataset

marco_ds = load_dataset("microsoft/ms_marco", "v1.1")
data_test = marco_ds['test']
ms_marco = data_test.select_columns(['passages', 'query', 'query_id'])
ms_marco

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

Dataset({
    features: ['passages', 'query', 'query_id'],
    num_rows: 9650
})

In [6]:
titles = []
queries = []
contexts = []
for index, example in enumerate(data_test):
    # Access individual fields like 'query', 'query_id', and 'passages'
    query = example['query']
    query_id = example['query_id']
    is_selected = example['passages']['is_selected']
    passage_text = example['passages']['passage_text']
    for i in range(len(is_selected)):
        titles.append("")
        if is_selected[i] == 1:
            queries.append([query])
            contexts.append(passage_text[i])
        else:
            queries.append([])
            contexts.append(passage_text[i])

print()


In [7]:
import pandas as pd
# from collections import defaultdict
  
dataset_for_eval_df = pd.DataFrame({'title' : titles,
                                    'context' : contexts,
                                  #  'query_id' : dataset_for_eval['id'],
                                    'queries' : queries})

dataset_for_eval_df.head()

,title,context,queries
0,,We have been feeding our back yard squirrels f...,[]
1,,"Critters cannot stand the smell of human hair,...",[]
2,,Spread some human hair around your vegetable a...,[does human hair stop squirrels]
3,,1 You can sprinkle blood meal around your gard...,[]
4,,Hair loss in squirrels can be caused by mange ...,[]


In [8]:
table = pa.Table.from_pandas(dataset_for_eval_df)

pq.write_table(table, 'processed_datasets/marco_processed.parquet')